In [1]:
#default_exp model.ms2

In [2]:
#hide
%reload_ext autoreload
%autoreload 2

# MS2 Spectrum Prediction Models

The models notebook implements deep learning models for the prediction of MS2 Spectra from peptide Sequences, using additional information such as charge state, collision energy and instrument. The components of the models are defined in the `building_block.ipynb` notebook. It is based on the interface defined in `model_interface.ipynb`, which defines standard interactions, such as loading data and training the model.

## Imports

In [3]:
#export
import torch
import pandas as pd
import numpy as np

from typing import List, Tuple, IO

from tqdm import tqdm

from alphabase.peptide.fragment import (
    init_fragment_by_precursor_dataframe, 
    update_sliced_fragment_dataframe, 
    get_sliced_fragment_dataframe, 
    get_charged_frag_types
)

from peptdeep.model.featurize import (
    get_batch_aa_indices, parse_instrument_indices, 
    get_batch_mod_feature
)

from peptdeep.settings import (
    global_settings as settings, 
    model_const
)


import peptdeep.model.model_interface as model_interface
import peptdeep.model.building_block as building_block

## Model Definitions

In the model definition classes below, all layers of the model are defined, utilizing the "building blocks" defined in `building_blocks.ipynb`

### Transformer Models

#### Pytorch Transformer

In [4]:
#export
class ModelMS2Transformer(torch.nn.Module):
    def __init__(self,
        num_frag_types,
        num_modloss_types=0,
        mask_modloss=True,
        dropout=0.1,
        nlayers=4,
        hidden=256,
        **kwargs,
    ):
        """
        Transformer model based on pytorch architecture for MS2 Spectrum prediction.
        Inputs are sequence, modification, charge, NCE (normalized collision energy) 
        and instrument.
        
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self._num_modloss_types = num_modloss_types
        self._num_non_modloss = num_frag_types-num_modloss_types
        self._mask_modloss = mask_modloss
        if num_modloss_types == 0:
            self._mask_modloss = True

        meta_dim = 8
        self.input_nn = building_block.Input_AA_Mod_PositionalEncoding(hidden-meta_dim)

        self.meta_nn = building_block.Meta_Embedding(meta_dim)
        
        self.hidden_nn = building_block.Hidden_Transformer(
            hidden, nlayers=nlayers, dropout=dropout
        )

        self.output_nn = building_block.Decoder_Linear(
            hidden,
            self._num_non_modloss,
        )
        
        if num_modloss_types > 0:
            self.modloss_nn = self._get_modloss_nn(hidden=hidden, dropout=dropout, num_modloss_types=num_modloss_types)
        else:
            self.modloss_nn = None

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):

        in_x = self.dropout(self.input_nn(
            aa_indices, mod_x
        ))
        meta_x = self.meta_nn(
            charges, NCEs, instrument_indices
        ).unsqueeze(1).repeat(1,in_x.size(1),1)
        in_x = torch.cat((in_x, meta_x),2)

        hidden_x = self.hidden_nn(in_x)
        hidden_x = self.dropout(hidden_x+in_x*0.2)

        out_x = self.output_nn(
            hidden_x
        )

        if self._num_modloss_types > 0:
            if self._mask_modloss:
                out_x = torch.cat((out_x, torch.zeros(
                    *out_x.size()[:2],self._num_modloss_types,
                    device=in_x.device
                )), 2)
            else:
                modloss_x = self.modloss_nn[0](
                    in_x
                ) + hidden_x
                modloss_x = self.modloss_nn[-1](
                    modloss_x
                )
                out_x = torch.cat((
                    out_x, modloss_x
                ),2)

        return out_x[:,3:,:]

    @staticmethod
    def _get_modloss_nn(hidden, dropout, num_modloss_types):
        """
        transformer model for transfer learning of fragments with modification losses
        """
        modloss_nn = torch.nn.ModuleList([
                    building_block.Hidden_Transformer(
                        hidden, nlayers=1, dropout=dropout
                    ),
                    building_block.Decoder_Linear(
                        hidden, num_modloss_types,
                    ),
                ])
        return modloss_nn

##### Test PyTorch Transformer

In [5]:
#hide


class SmallInputforTest():
    def __init__(self):
        torch.manual_seed(1337)
        import numpy as np
        self.sequence = 'AAAAAAAA'
        self.max_instrument_num = 4
        self.mod_feature_size=len(model_const['mod_elements'])
        self.aa_indices = torch.LongTensor(get_batch_aa_indices([self.sequence]))
        self.charges = torch.tensor([[2]])
        self.instrument_indices = torch.LongTensor([1])
        self.NCEs = torch.tensor([[0.3]])
        self.mod_x = torch.zeros((1, self.aa_indices.size(1), self.mod_feature_size))
        

def test_that_output_dimensions_are_as_expected(msms_model):
    test_input = SmallInputforTest()
    msms_model.eval()
    out = msms_model(test_input.aa_indices, test_input.mod_x, test_input.charges, test_input.NCEs, test_input.instrument_indices)
    assert out.size() == (1,len(test_input.sequence)-1,2)
    print(out)
    
model1 = ModelMS2Transformer(2, 1, mask_modloss=False)
test_that_output_dimensions_are_as_expected(model1)

model2 = ModelMS2Transformer(2, 1, mask_modloss=True)
test_that_output_dimensions_are_as_expected(model2)


tensor([[[ 0.3587, -0.1867],
         [ 0.4103, -0.2554],
         [ 0.4475, -0.2975],
         [ 0.4613, -0.3075],
         [ 0.4498, -0.3135],
         [ 0.4212, -0.3193],
         [ 0.4311, -0.3754]]], grad_fn=<SliceBackward0>)
tensor([[[-0.0770,  0.0000],
         [-0.0555,  0.0000],
         [-0.0159,  0.0000],
         [ 0.0321,  0.0000],
         [ 0.0602,  0.0000],
         [ 0.0463,  0.0000],
         [-0.0423,  0.0000]]], grad_fn=<SliceBackward0>)


#### Huggingface Bert Transformer

In [6]:
#export
class ModelMS2Bert(torch.nn.Module):
    def __init__(self,
        num_frag_types,
        num_modloss_types=0,
        mask_modloss=True,
        dropout=0.1,
        nlayers=4,
        hidden=256,
        output_attentions=False,
        **kwargs,
    ):
        """
        Transformer model based on HuggingFace's BertEncoder 
        for MS2 Spectrum prediction. Inputs are sequence,
        modification, charge, NCE (normalized collision energy) and instrument
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self._num_modloss_types = num_modloss_types
        self._num_non_modloss = num_frag_types-num_modloss_types
        self._mask_modloss = mask_modloss
        if num_modloss_types == 0:
            self._mask_modloss = True

        meta_dim = 8
        self.input_nn = building_block.Input_AA_Mod_PositionalEncoding(hidden-meta_dim)

        self.meta_nn = building_block.Meta_Embedding(meta_dim)

        self._output_attentions = output_attentions
        self.hidden_nn = building_block.Hidden_HFace_Transformer(
            hidden, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = building_block.Decoder_Linear(
            hidden,
            self._num_non_modloss,
        )
        
        if num_modloss_types > 0:
            # for transfer learning of modloss frags
            self.modloss_nn = torch.nn.ModuleList([
                building_block.Hidden_HFace_Transformer(
                    hidden, nlayers=1, dropout=dropout,
                    output_attentions=output_attentions
                ),
                building_block.Decoder_Linear(
                    hidden, num_modloss_types,
                ),
            ])
        else:
            self.modloss_nn = None

    @property
    def output_attentions(self):
        return self._output_attentions
        
    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val
        self.modloss_nn[0].output_attentions = val

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):

        in_x = self.dropout(self.input_nn(
            aa_indices, mod_x
        ))
        meta_x = self.meta_nn(
            charges, NCEs, instrument_indices
        ).unsqueeze(1).repeat(1,in_x.size(1),1)
        in_x = torch.cat((in_x, meta_x),2)

        hidden_x = self.hidden_nn(in_x)
        if self.output_attentions:
            self.attentions = hidden_x[1]
        else:
            self.attentions = None
        hidden_x = self.dropout(hidden_x[0]+in_x*0.2)

        out_x = self.output_nn(
            hidden_x
        )

        self.modloss_attentions = None
        if self._num_modloss_types > 0:
            if self._mask_modloss:
                out_x = torch.cat((out_x, torch.zeros(
                    *out_x.size()[:2],self._num_modloss_types,
                    device=in_x.device
                )), 2)
            else:
                modloss_x = self.modloss_nn[0](
                    in_x
                )
                if self.output_attentions:
                    self.modloss_attentions = modloss_x[-1]
                modloss_x = modloss_x[0] + hidden_x
                modloss_x = self.modloss_nn[-1](
                    modloss_x
                )
                out_x = torch.cat((
                    out_x, modloss_x
                ),2)

        return out_x[:,3:,:]

#### Test Bert Transformer

In [7]:
#hide
def print_attentions(msms_model):
    test_input = SmallInputforTest()
    msms_model.output_attentions = True
    msms_model(test_input.aa_indices, test_input.mod_x, test_input.charges, test_input.NCEs, test_input.instrument_indices)
    print((
        len(msms_model.attentions), msms_model.attentions[0].size(),
        len(msms_model.modloss_attentions), msms_model.modloss_attentions[0].size()
    )
    )


msms_model = ModelMS2Bert(2, 1, mask_modloss=False)
test_that_output_dimensions_are_as_expected(msms_model)

print_attentions(msms_model)


tensor([[[-0.3243, -0.7804],
         [-0.3156, -0.7880],
         [-0.3168, -0.8275],
         [-0.3246, -0.9223],
         [-0.3048, -1.0358],
         [-0.2640, -1.1187],
         [-0.3313, -0.4993]]], grad_fn=<SliceBackward0>)
(4, torch.Size([1, 8, 10, 10]), 1, torch.Size([1, 8, 10, 10]))


### LSTM Model

In [8]:
#export
class ModelMS2LSTM(torch.nn.Module):
    def __init__(self,
        num_frag_types,
        num_modloss_types=0,
        mask_modloss=True,
        dropout=0.1,
        **kwargs,
    ):
        """
        LSTM Model for MS2 Spectrum prediction using the architecture of the `pdeep` Model. Inputs are sequence,
        modification, charge, NCE (normalized collision energy) and instrument
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self._num_modloss_types = num_modloss_types
        self._num_non_modloss = num_frag_types-num_modloss_types
        self._mask_modloss = mask_modloss
        if num_modloss_types == 0:
            self._mask_modloss = True

        BiRNN = True
        hidden=512
        hidden_rnn_layer=2

        self.input_nn = building_block.InputAALSTM_cat_Meta(hidden)

        self.hidden_nn = building_block.SeqLSTM(
            hidden, hidden, rnn_layer=hidden_rnn_layer, 
            bidirectional=BiRNN
        )

        self.output_nn = building_block.OutputLSTM_cat_Meta(
            hidden,
            self._num_non_modloss,
        )

        if num_modloss_types:
            # for transfer learning of modloss frags
            self.modloss_nn = torch.nn.ModuleList([
                building_block.SeqLSTM(
                    hidden, hidden, 
                    rnn_layer=1, bidirectional=BiRNN
                ),
                building_block.SeqLSTM(
                    hidden, num_modloss_types,
                    rnn_layer=1, bidirectional=False
                ),
            ])
        else:
            self.modloss_nn = None

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):

        in_x = self.input_nn(
            aa_indices, mod_x, 
            charges, NCEs, instrument_indices
        )
        in_x = self.dropout(in_x)

        hidden_x = self.hidden_nn(in_x)
        hidden_x = self.dropout(hidden_x)

        out_x = self.output_nn(
            hidden_x, 
            charges, NCEs, instrument_indices
        )

        # modloss is mainly only for Phospho@S/T
        if self._num_modloss_types > 0:
            if self._mask_modloss:
                out_x = torch.cat((out_x, torch.zeros(
                    *out_x.size()[:2],self._num_modloss_types,
                    device=in_x.device
                )), 2)
            else:
                modloss_x = self.modloss_nn[0](
                    in_x
                ) + hidden_x
                modloss_x = self.modloss_nn[-1](
                    modloss_x
                )
                out_x = torch.cat((
                    out_x, modloss_x
                ),2)

        return out_x[:,3:,:]


#legacy
ModelMS2pDeep = ModelMS2LSTM

In [9]:
#hide

msms_model =  ModelMS2LSTM(2, 1, mask_modloss=False)
test_that_output_dimensions_are_as_expected(msms_model)

tensor([[[0.7883, 0.4976],
         [0.8469, 0.4712],
         [0.8717, 0.4456],
         [0.8818, 0.4204],
         [0.8841, 0.3938],
         [0.8799, 0.3610],
         [0.8665, 0.3113]]], grad_fn=<SliceBackward0>)


In [10]:
#export
class IntenAwareLoss(torch.nn.Module):
    def __init__(self, base_weight=0.2):
        super().__init__()
        self.w = base_weight

    def forward(self, pred, target):
        return torch.mean(
            (target+self.w)*torch.abs(target-pred)
        )

## Model Pipeline defining the pDeepModel for MS/MS prediction
The final model which is used to interact with the data is defined below, utilizing the `ModelInterface` class defined in `model_interface.ipynb` 


In [11]:
#export
mod_feature_size = len(model_const['mod_elements'])
max_instrument_num = model_const['max_instrument_num']
frag_types = settings['model']['frag_types']
max_frag_charge = settings['model']['max_frag_charge']
num_ion_types = len(frag_types)*max_frag_charge

class pDeepModel(model_interface.ModelInterface):
    def __init__(self,
        charged_frag_types = get_charged_frag_types(
            frag_types, max_frag_charge
        ),
        dropout=0.1,
        mask_modloss=True,
        modloss_type='modloss',
        model_class:torch.nn.Module=ModelMS2Bert,
        device:str='gpu',
        **kwargs, #model params
    ):
        """
        class to predict MS2 Data from precursor dataframe
        """
        super().__init__(device=device)
        self.charged_frag_types = charged_frag_types
        self._get_modloss_frags(modloss_type)

        self.charge_factor = 0.1
        self.NCE_factor = 0.01
        self.build(
            model_class, 
            num_frag_types = len(self.charged_frag_types),
            num_modloss_types = len(self._modloss_frag_types),
            mask_modloss=mask_modloss,
            dropout=dropout,
            **kwargs, # other model params
        )

        self.loss_func = torch.nn.L1Loss()
        self.min_inten = 1e-4

    def _get_modloss_frags(self, modloss='modloss'):
        self._modloss_frag_types = []
        for i,frag in enumerate(self.charged_frag_types):
            if modloss in frag:
                self._modloss_frag_types.append(i)

    def _prepare_train_data_df(self, 
        precursor_df:pd.DataFrame,
        fragment_intensity_df:pd.DataFrame=None,
    ):
        self.frag_inten_df = fragment_intensity_df[self.charged_frag_types]
        # if np.all(precursor_df['nce'].values > 1):
        #     precursor_df['nce'] = precursor_df['nce']*self.NCE_factor

    def _check_predict_in_order(self, precursor_df: pd.DataFrame):
        pass

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
        reference_frag_df:pd.DataFrame=None,
    ):
        if reference_frag_df is None and precursor_df.nAA.is_monotonic:
            self._predict_in_order = True
            
            if 'frag_start_idx' in precursor_df.columns:
                precursor_df.drop(
                    columns=['frag_start_idx','frag_end_idx'],
                    inplace=True
                )
        else:
            self._predict_in_order = False
        
        self.predict_df = init_fragment_by_precursor_dataframe(
            precursor_df, self.charged_frag_types, 
            reference_fragment_df=reference_frag_df, 
            dtype=np.float32
        )

        # if np.all(precursor_df['nce'].values > 1):
        #     precursor_df['nce'] = precursor_df['nce']*self.NCE_factor

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame, 
        **kwargs,
    ) -> Tuple[torch.Tensor]:
        aa_indices = self._as_tensor(
            get_batch_aa_indices(
                batch_df['sequence'].values.astype('U')
            ), 
            dtype=torch.long
        )

        mod_x = self._as_tensor(
            get_batch_mod_feature(
                batch_df
            )
        )

        charges = self._as_tensor(
            batch_df['charge'].values
        ).unsqueeze(1)*self.charge_factor

        nces = self._as_tensor(
            batch_df['nce'].values
        ).unsqueeze(1)*self.NCE_factor

        instrument_indices = self._as_tensor(
            parse_instrument_indices(batch_df['instrument']),
            dtype=torch.long
        )
        return aa_indices, mod_x, charges, nces, instrument_indices

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
        fragment_intensity_df:pd.DataFrame=None
    ) -> torch.Tensor:
        return self._as_tensor(
            get_sliced_fragment_dataframe(
                fragment_intensity_df, 
                batch_df[
                    ['frag_start_idx','frag_end_idx']
                ].values
            ).values
        ).view(-1, 
            batch_df.nAA.values[0]-1, 
            len(self.charged_frag_types)
        )

    def _set_batch_predict_data(self, 
        batch_df: pd.DataFrame, 
        predicts:np.ndarray,
        **kwargs,
    ):
        apex_intens = predicts.reshape((len(batch_df), -1)).max(axis=1)
        apex_intens[apex_intens<=0] = 1
        predicts /= apex_intens.reshape((-1,1,1))
        predicts[predicts<self.min_inten] = 0.0
        if self._predict_in_order:
            self.predict_df.values[
                batch_df.frag_start_idx.values[0]:
                batch_df.frag_end_idx.values[-1], 
            :] = predicts.reshape(
                    (-1, len(self.charged_frag_types))
                )
        else:
            update_sliced_fragment_dataframe(
                self.predict_df,
                predicts.reshape(
                    (-1, len(self.charged_frag_types))
                ),
                batch_df[
                    ['frag_start_idx','frag_end_idx']
                ].values,
            )

    def train_with_warmup(self,
        precursor_df: pd.DataFrame, 
        fragment_intensity_df, 
        *, 
        batch_size=1024, 
        epoch=10,
        warmup_epoch=5,
        lr = 1e-4,
        verbose=False, 
        verbose_each_epoch=False,
        **kwargs
    ):
        return super().train_with_warmup(
            precursor_df, 
            fragment_intensity_df=fragment_intensity_df,
            batch_size=batch_size, 
            epoch=epoch,
            warmup_epoch=warmup_epoch,
            lr=lr,
            verbose=verbose, 
            verbose_each_epoch=verbose_each_epoch,
            **kwargs
        )

    def train(self, 
        precursor_df: pd.DataFrame, 
        fragment_intensity_df, 
        *, 
        batch_size=1024, 
        epoch=20,
        warmup_epoch=0,
        verbose=False, 
        verbose_each_epoch=False,
        **kwargs
    ):
        return super().train(
            precursor_df, 
            fragment_intensity_df=fragment_intensity_df,
            batch_size=batch_size, 
            epoch=epoch, 
            warmup_epoch=warmup_epoch,
            verbose=verbose, 
            verbose_each_epoch=verbose_each_epoch,
            **kwargs
        )

    def predict(self, 
        precursor_df: pd.DataFrame, 
        *, 
        batch_size=1024, 
        verbose=False, 
        reference_frag_df=None,
        **kwargs
    ) -> pd.DataFrame:
        return super().predict(
            precursor_df, 
            batch_size=batch_size, 
            verbose=verbose, 
            reference_frag_df=reference_frag_df, 
            **kwargs
        )

    def bootstrap_nce_search(self, 
        psm_df:pd.DataFrame, 
        fragment_intensity_df:pd.DataFrame,
        nce_first=15, nce_last=45, nce_step=3,
        instrument = 'Lumos',
        charged_frag_types:List = None,
        metric = 'PCC>0.9', # or 'median PCC'
        max_psm_subset = 3000,
        n_bootstrap = 3,
        callback = None
    ):
        nce_list = []
        for i in range(n_bootstrap):
            nce, instrument = self.grid_nce_search(
                psm_df, fragment_intensity_df,
                nce_first, nce_last, nce_step,
                [instrument],
                charged_frag_types,
                metric, max_psm_subset, n_bootstrap,
                callback
            )
            nce_list.append(nce)
        return np.median(nce_list), instrument

    def grid_nce_search(self,
        psm_df:pd.DataFrame, 
        fragment_intensity_df:pd.DataFrame,
        nce_first=15, nce_last=45, nce_step=3,
        search_instruments = ['Lumos'],
        charged_frag_types:List = None,
        metric = 'PCC>0.9', # or 'median PCC'
        max_psm_subset = 1000000,
        callback = None
    ):
        if len(psm_df) > max_psm_subset:
            psm_df = psm_df.sample(max_psm_subset).copy()
        best_pcc = -1
        best_nce = 0.
        best_instrument = None
        if 'median' in metric:
            metric_row = '50%'
        else:
            metric_row = '>0.90'
        for inst in search_instruments:
            for nce in np.arange(nce_first, nce_last+nce_step, nce_step):
                psm_df['nce'] = nce
                psm_df['instrument'] = inst
                predict_inten_df = self.predict(
                    psm_df, 
                    reference_frag_df=fragment_intensity_df
                )
                df, metrics = calc_ms2_similarity(
                    psm_df,
                    predict_inten_df, 
                    fragment_intensity_df,
                    charged_frag_types=charged_frag_types,
                    metrics=['PCC']
                )
                pcc = metrics.loc[metric_row, 'PCC']
                if pcc > best_pcc:
                    best_pcc = pcc
                    best_nce = nce
                    best_instrument = inst
        return best_nce, best_instrument


## Utility Functions

### Preparation of input data

In [12]:
#export
def normalize_training_intensities(
    train_df:pd.DataFrame, 
    frag_intensity_df:pd.DataFrame
)->Tuple[pd.DataFrame, pd.DataFrame]:
    """Normalize the intensities to 0-1 values for MS2 model training.

    Args:
        train_df (pd.DataFrame): training psm dataframe
        frag_intensity_df (pd.DataFrame): training intensity dataframe

    Returns:
        pd.DataFrame: normalized training psm dataframe
        pd.DataFrame: normalized training intensity dataframe
    """
    new_frag_intens_list = []
    new_frag_lens = []
    for i, (frag_start_idx, frag_end_idx) in enumerate(
        train_df[['frag_start_idx','frag_end_idx']].values
    ):
        intens = frag_intensity_df.values[frag_start_idx:frag_end_idx]
        new_frag_lens.append(len(intens))
        max_inten = np.max(intens)
        if max_inten > 0:
            intens /= max_inten
        new_frag_intens_list.append(intens)
    indices = np.zeros(len(new_frag_lens)+1, dtype=np.int64)
    indices[1:] = new_frag_lens
    indices = np.cumsum(indices)
    train_df['frag_start_idx'] = indices[:-1]
    train_df['frag_end_idx'] = indices[1:]

    frag_df = pd.DataFrame(
        data=np.concatenate(new_frag_intens_list, axis=0), 
        columns=frag_intensity_df.columns
    )
    return train_df, frag_df
        

### Different similarity metrics, moved to utils

In [13]:
#export
from peptdeep import utils

#legacy
pearson = utils.pearson_correlation


#legacy
spectral_angle = utils.spectral_angle


#legacy
spearman = utils.spearman_correlation

#legacy
add_cutoff_metric = utils.add_cutoff_metric

#legacy
calc_ms2_similarity = utils.add_ms2_similarity_columns


## Unit Tests
### Test Small Workflow

In [26]:
#hide

def generate_small_precursor_df():
    repeat = 10
    precursor_df = pd.DataFrame({
        'sequence': ['AGHCEWQMKYR']*repeat,
        'mods': ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat,
        'mod_sites': ['0;4;8']*repeat,
        'nAA': [11]*repeat,
        'nce': [20]*repeat,
        'instrument': 'QE',
        'charge': np.arange(1,repeat+1),
        'frag_start_idx':np.arange(10, dtype=int)*10,
        'frag_end_idx':np.arange(10, dtype=int)*10+10,
    })
    return precursor_df

precursor_df = generate_small_precursor_df()
display(precursor_df)

,sequence,mods,mod_sites,nAA,nce,instrument,charge,frag_start_idx,frag_end_idx
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,1,0,10
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,2,10,20
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,3,20,30
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,4,30,40
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,5,40,50
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,6,50,60
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,7,60,70
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,8,70,80
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,9,80,90
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,90,100


In [28]:
#hide

def generate_small_fragment_intensity_df():
    fragment_intensity_df = pd.DataFrame({
    'b':np.arange(100)/100, 
    'y_modloss':np.arange(100)/100
})
    return fragment_intensity_df

fragment_intensity_df = generate_small_fragment_intensity_df()
display(fragment_intensity_df)

,b,y_modloss
0,0.00,0.00
1,0.01,0.01
2,0.02,0.02
3,0.03,0.03
4,0.04,0.04
...,...,...
95,0.95,0.95
96,0.96,0.96
97,0.97,0.97
98,0.98,0.98


### Train an MSMS model

In [29]:
#hide

def train_pdeep_model():
    pdeep_model = pDeepModel()
    pdeep_model.charged_frag_types=['b','y_modloss']
    pdeep_model.build(
        ModelMS2pDeep,
        num_frag_types = 2,
        num_modloss_types = 1,
        dropout=0.2,
        lr=1e-3
    )
    pdeep_model.train(precursor_df, epoch=5, batch_size=3, fragment_intensity_df=fragment_intensity_df)
    pdeep_model.train_with_warmup(precursor_df, epoch=5, batch_size=3, warmup_epoch=3, fragment_intensity_df=fragment_intensity_df)
    return pdeep_model

trained_model = train_pdeep_model()


### Predict from the model

In [30]:
#hide
predict_inten_df = trained_model.predict(precursor_df,reference_frag_df=fragment_intensity_df)
predict_inten_df.max()
display(precursor_df)

,sequence,mods,mod_sites,nAA,nce,instrument,charge,frag_start_idx,frag_end_idx
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,1,0,10
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,2,10,20
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,3,20,30
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,4,30,40
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,5,40,50
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,6,50,60
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,7,60,70
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,8,70,80
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,9,80,90
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,90,100


### Metrics

In [36]:
#hide

def add_similarity_columns_and_check_that_correlations_behave_as_expected():
    _frag_inten_df = fragment_intensity_df.copy()
    _frag_inten_df.iloc[0,1] = 0.22

    df, metrics_describ = calc_ms2_similarity(
        precursor_df, fragment_intensity_df, 
        _frag_inten_df, ['b'],
        spc_top_k=0
    )
    assert df.SPC.values[0] == 1
    df, metrics_describ = calc_ms2_similarity(
        precursor_df, fragment_intensity_df, 
        _frag_inten_df, ['b','y_modloss'],
        spc_top_k=2
    )
    assert df.SPC.values[0] == -1
    display(precursor_df)
    display(metrics_describ)
    return precursor_df

precursor_df = add_similarity_columns_and_check_that_correlations_behave_as_expected()


,sequence,mods,mod_sites,nAA,nce,instrument,charge,frag_start_idx,frag_end_idx,PCC,COS,SA,SPC
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,1,0,10,0.248708,0.735389,0.526001,-1.0
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,2,10,20,1.000000,1.000000,1.000000,1.0
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,3,20,30,1.000000,1.000000,1.000000,1.0
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,4,30,40,1.000000,1.000000,1.000000,1.0
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,5,40,50,1.000000,1.000000,1.000000,1.0
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,6,50,60,1.000000,1.000000,1.000000,1.0
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,7,60,70,1.000000,1.000000,1.000000,1.0
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,8,70,80,1.000000,1.000000,1.000000,1.0
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,9,80,90,1.000000,1.000000,1.000000,1.0
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,45,Lumos,10,90,100,1.000000,1.000000,1.000000,1.0


,PCC,COS,SA,SPC
count,10.000000,10.000000,10.000000,10.000000
mean,0.924871,0.973539,0.952600,0.800000
std,0.237579,0.083677,0.149892,0.632456
min,0.248708,0.735389,0.526001,-1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000
>0.90,0.900000,0.900000,0.900000,0.900000
>0.75,0.900000,0.900000,0.900000,0.900000


In [37]:
#hide
def get_matching_nce_and_model(precursor_df, fragment_intensity_df):
    print(trained_model.grid_nce_search(precursor_df, fragment_intensity_df))

get_matching_nce_and_model(precursor_df, fragment_intensity_df)

(15, 'Lumos')


In [38]:
#hide
def get_number_parameters():
    trained_model = pDeepModel(mask_modloss=False)
    print(trained_model.get_parameter_num())

get_number_parameters()

3988974
